In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd  # Import pandas library

base_url = "https://www.tradingview.com/symbols/NASDAQ-AAPL/ideas/"
sort_option = "?sort=recent"
max_pages = 30  # Adjust the number of pages you want to scrape

data_list = []

for page in range(1, max_pages + 1):
    url = f"{base_url}page-{page}/{sort_option}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    timestamp_elements = soup.find_all("span", class_="tv-card-stats__time", attrs={"data-timestamp": True})
    a_elements = soup.find_all("a", class_="tv-widget-idea__cover-link js-widget-idea__popup")
    info_rows = soup.find_all("div", class_="tv-widget-idea__info-row")
    username_elements = soup.find_all("span", class_="tv-card-user-info__name")
    
    if not timestamp_elements or not a_elements or not info_rows or not username_elements:
        break
    
    for timestamp_element, a_element, row, username_element in zip(timestamp_elements, a_elements, info_rows, username_elements):
        timestamp = timestamp_element["data-timestamp"]
        img_element = a_element.find("img")
        alt_text = img_element["alt"]
        data_href = a_element["data-href"]
        
        label_element = row.find("span", class_="badge-PlSmolIm size-medium-PlSmolIm badge-idea-ZleujXqe type-short-ZleujXqe size-medium-ZleujXqe variant-primary-ZleujXqe with-icon-ZleujXqe tv-widget-idea__label") or \
                        row.find("span", class_="badge-PlSmolIm size-medium-PlSmolIm badge-idea-ZleujXqe type-long-ZleujXqe size-medium-ZleujXqe variant-primary-ZleujXqe with-icon-ZleujXqe tv-widget-idea__label")
        
        if label_element:
            label_text = label_element.find("span", class_="content-PlSmolIm badge-idea-content-ZleujXqe").get_text()
        else:
            label_text = "N/A"
        
        username = username_element.get_text()
        
        chart_url = f"https://www.tradingview.com{data_href}"
        response = requests.get(chart_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        description_element = soup.find("div", class_="tv-chart-view__description")
        if description_element:
            description_text = description_element.get_text()
        else:
            description_text = "Description not found on the page."
        
        data_list.append((timestamp, alt_text, label_text, description_text, username))

# Create a DataFrame from the collected data
df = pd.DataFrame(data_list, columns=["Timestamp", "Alt Text", "Label", "Description", "Username"])

# Print the DataFrame
print(df)


        Timestamp                                           Alt Text  Label  \
0    1692129257.0            AAPL: Apple -> Correction Already Over?   Long   
1    1692298346.0  AAPL: AAPL, Crucial Trend-Dynamics, Timeframes...    N/A   
2    1692036887.0                    AAPL: ✅APPLE LONG FROM SUPPORT🚀   Long   
3    1692314572.0                 AAPL: Target 159 in September 2023  Short   
4    1691951238.0                                         AAPL: Aapl   Long   
..            ...                                                ...    ...   
535  1684900691.0  AAPL: Polygon Integrates De.Fi's Web3 Antiviru...    N/A   
536  1684900217.0  AAPL: Bank of America: Crypto markets have lim...    N/A   
537  1684899864.0                        AAPL: BTC strategy analysis    N/A   
538  1684867843.0                                      AAPL: Aapl 🍎   Short   
539  1684866354.0              AAPL: Huge rising wedge - likely drop  Short   

                                           Descript

In [2]:
   
# Create a DataFrame from the collected data
df = pd.DataFrame(data_list, columns=["Timestamp", "Alt Text", "Label", "Description", "Username"])

# Convert Timestamp to datetime
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="s")

# Assign appropriate data types to other columns
data_types = {
    "Alt Text": str,
    "Label": str,
    "Description": str,
    "Username": str
}

df = df.astype(data_types)

C:\Users\Chris\AppData\Local\Temp\ipykernel_10420\302196541.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="s")


In [3]:
df.head()

,Timestamp,Alt Text,Label,Description,Username
0,2023-08-15 19:54:40,AAPL: Apple -> Correction Already Over?,Long,"Hello Traders and Investors, \r\nmy name is Ph...",basictradingtv
1,2023-08-17 18:52:48,"AAPL: AAPL, Crucial Trend-Dynamics, Timeframes...",N/A,\n\nHello There! \n\r\nWelcome to my video id...,VincePrince
2,2023-08-14 18:14:24,AAPL: ✅APPLE LONG FROM SUPPORT🚀,Long,\n\n\n\n\n\n✅APPLE is trading in an \r\nUptren...,ProSignalsFx
3,2023-08-17 23:23:44,AAPL: Target 159 in September 2023,Short,\n\n In my method I can...,JerryManders
4,2023-08-13 18:27:12,AAPL: Aapl,Long,"Weekly chart is hideous, definitely expectin...",ContraryTrader


In [4]:
from textblob import TextBlob


# Function to perform sentiment analysis using TextBlob
def analyze_sentiment(description):
    analysis = TextBlob(description)
    sentiment_score = analysis.sentiment.polarity
    if sentiment_score > 0:
        return "Positive"
    elif sentiment_score < 0:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to the 'Description' column
df['Sentiment'] = df['Description'].apply(analyze_sentiment)

# Encode sentiment labels into numerical values
sentiment_mapping = {"Positive": 1, "Neutral": 0, "Negative": -1}
df['Sentiment_Encoded'] = df['Sentiment'].map(sentiment_mapping)

# Encode label values into numerical values
label_mapping = {"Long": 1, "N/A": 0, "Short": -1}
df['Label_Encoded'] = df['Label'].map(label_mapping)

# Calculate correlation between 'Label_Encoded' and 'Sentiment_Encoded'
correlation = df['Label_Encoded'].corr(df['Sentiment_Encoded'])

# Print the correlation value
print("Correlation between Label and Sentiment:", correlation)

Correlation between Label and Sentiment: 0.03627804004582833


In [5]:
df.groupby(['Label', 'Sentiment'])['Timestamp'].count()

Label  Sentiment
Long   Negative      26
       Neutral       15
       Positive     105
N/A    Negative      47
       Neutral       28
       Positive     174
Short  Negative      32
       Neutral       14
       Positive      99
Name: Timestamp, dtype: int64

In [6]:
pd.set_option('display.max_colwidth', None)

df[df['Label'] == 'Short']['Description']

3      \n\n                        In my method I can confirm when a B wave has completed. For AAPL, failure to extend to 201 and 204 signals B wave has finished, and we will get confirmation that C wave has started with completion of wave (a) of C down. My wave (a) target is 159. \n\r\nThe solid black trajectory is my main path: \n\r\n-Drop to 179-185 (targeting lower range) \r\n-Then bounce to 186-194 (targeting lower range) \r\n-Finally explosive drop to 159 (159 is only 0.5 retracement of the B wave, could go lower to 0.618 or 0.786 which would be 152, 140 respectively). Expecting 159 though to test breakout level from inverse head and shoulders that formed at base of B-wave. \n\r\nTime to target in main path is end of August \n\r\n------------------------------------- \n\r\nThe dashed black path is my alt path: \n\r\n-If we see bounce here around 183-186 then it would likely test low-mid 190s for resistance \r\n-Then drop to 173-177 \r\n-Then one more shallow bounce to 179-183 \r\

In [7]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Set display option to show full content of columns
pd.set_option('display.max_colwidth', None)

# Load NLTK stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



# Tokenize and preprocess the description text
def preprocess_text(text):
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    return filtered_words

# Apply preprocessing to the Description column
df['Processed_Description'] = df['Description'].apply(preprocess_text)

# Count keywords
keyword_counter = Counter()
for keywords in df['Processed_Description']:
    keyword_counter.update(keywords)

# Display the most common keywords
most_common_keywords = keyword_counter.most_common(10)
print("Most common keywords:", most_common_keywords)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Most common keywords: [('apple', 466), ('market', 438), ('price', 429), ('trading', 366), ('aapl', 259), ('see', 201), ('stock', 195), ('time', 186), ('wave', 174), ('may', 172)]


In [8]:
df[['Description','Processed_Description']].head()

,Description,Processed_Description
0,"Hello Traders and Investors, \r\nmy name is Philip and today I will provide a free and educational multi-timeframe technical analysis of Apple💪 \n\r\nStarting on the monthly timeframe you can see that after Apple broke out of the clear triangle formation in confluence with the bullish moving averages, Apple created a strong rally of 30% towards the upside, breaking major resistance. \n\r\nOn the weekly timeframe you can see that Apple is already approching previous resistance which could be acting as support and after the retest of the 0.382 fibonacci retracement level we could see at least a short term bullish bounce. \n\r\nBut Apple stock is still creating bearish market structure so there is also a chance that Apple stock will just break below the current support level - If you are looking for longs though I would simply wait for a break aboce the bearish daily trendline and then enter a long position. \n\nKeep in mind: Don't get caught up in short term moves and always look at the long term picture; building wealth is a marathon and not a quick sprint📈 \n\r\nThank you for watching and I will see you tomorrow! \n\nMy previous analysis of this asset: \n\n\n\n\n03:42\n\n","[hello, traders, investors, name, philip, today, provide, free, educational, technical, analysis, starting, monthly, timeframe, see, apple, broke, clear, triangle, formation, confluence, bullish, moving, averages, apple, created, strong, rally, 30, towards, upside, breaking, major, resistance, weekly, timeframe, see, apple, already, approching, previous, resistance, could, acting, support, retest, fibonacci, retracement, level, could, see, least, short, term, bullish, bounce, apple, stock, still, creating, bearish, market, structure, also, chance, apple, stock, break, current, support, level, looking, longs, though, would, simply, wait, break, aboce, bearish, daily, trendline, enter, long, position, keep, mind, get, caught, short, term, moves, always, look, long, term, picture, building, wealth, marathon, ...]"
1,"\n\nHello There! \n\r\nWelcome to my video idea about AAPL on the 4-hour-, daily-, and weekly timeframe perspectives. AAPL is a major stock asset in the stock market with almost nearly $3 Billion in market-cap being the biggest market-cap stock within the stock market. \n\r\nIn my video I lift the secret behind the recent massive dump towards the downside and what should be considered with the dynamics in correlation with the levels that Apple approaches now. Apple is now in a crucial decisive movement with the bearish trend to either continue or reverse. \n\r\nWe will watch the symbol closely and cover any important changes in the dynamics. Especially within the next times and major changes within the market ongoing it will be an exciting dynamic when also considering a potential positioning in the market in the long- or short direction. \n\nIn this manner, it will be greatly appreciated when you support my video. Have a good day and many lucks in trading. \n\r\nVP\n \n","[hello, welcome, video, idea, aapl, weekly, timeframe, perspectives, aapl, major, stock, asset, stock, market, almost, nearly, 3, billion, biggest, stock, within, stock, market, video, lift, secret, behind, recent, massive, dump, towards, downside, considered, dynamics, correlation, levels, apple, approaches, apple, crucial, decisive, movement, bearish, trend, either, continue, reverse, watch, symbol, closely, cover, important, changes, dynamics, especially, within, next, times, major, changes, within, market, ongoing, exciting, dynamic, also, considering, potential, positioning, market, short, direction, manner, greatly, appreciated, support, video, good, day, many, lucks, trading, vp]"
2,\n\n\n\n\n\n✅APPLE is trading in an \r\nUptrend and the stock \r\nWas making a bearish correction \r\nBut now the price has reached a \r\nSupport level of 175.00$ from \r\nWhere I think we will see \r\nA bullish rebound \nLONG🚀 \n\n\r\n✅Like and subscribe 

In [9]:
import yfinance as yf 
from datetime import datetime
# Define the ticker symbol (AAPL for Apple)
ticker = "AAPL"

# Fetch historical data using yfinance
APPL=yf.download(ticker, start=df['Timestamp'].min(),end=df['Timestamp'].max())

APPL=APPL.reset_index()
APPL.info()

[*********************100%%**********************]  1 of 1 completed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       61 non-null     datetime64[ns]
 1   Open       61 non-null     float64       
 2   High       61 non-null     float64       
 3   Low        61 non-null     float64       
 4   Close      61 non-null     float64       
 5   Adj Close  61 non-null     float64       
 6   Volume     61 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 3.5 KB


In [14]:
# Convert Timestamp to datetime
from pandas.tseries.offsets import BDay
df["Timestamp"]=pd.to_datetime(df["Timestamp"])
df['Date'] = df['Timestamp'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
# Adjust the date to the next business day if necessary
df['Adjusted_Date'] = df['Date'].apply(lambda x: x if x.weekday() < 4 else x + BDay(1))
df[['Date','Adjusted_Date']].sort_values(by='Date')

,Date,Adjusted_Date
539,2023-05-23,2023-05-23
538,2023-05-23,2023-05-23
536,2023-05-24,2023-05-24
524,2023-05-24,2023-05-24
525,2023-05-24,2023-05-24
...,...,...
3,2023-08-17,2023-08-17
1,2023-08-17,2023-08-17
117,2023-08-18,2023-08-21
9,2023-08-18,2023-08-21


In [17]:
APPL[['Date','Adj Close']]
for days in [1, 3, 5, 10]:
    APPL[f'Return_{days}_Days'] = APPL['Adj Close'].shift(-days)
APPL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            61 non-null     datetime64[ns]
 1   Open            61 non-null     float64       
 2   High            61 non-null     float64       
 3   Low             61 non-null     float64       
 4   Close           61 non-null     float64       
 5   Adj Close       61 non-null     float64       
 6   Volume          61 non-null     int64         
 7   Return_1_Days   60 non-null     float64       
 8   Return_3_Days   58 non-null     float64       
 9   Return_5_Days   56 non-null     float64       
 10  Return_10_Days  51 non-null     float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 5.4 KB


In [25]:
merged_df = df.merge(APPL, right_on='Date', left_on='Adjusted_Date', how='left')
merged_df.sort_values(by='Date_x')

,Timestamp,Alt Text,Label,Description,Username,Sentiment,Sentiment_Encoded,Label_Encoded,Processed_Description,Date_x,...,Open,High,Low,Close,Adj Close,Volume,Return_1_Days,Return_3_Days,Return_5_Days,Return_10_Days
539,2023-05-23 18:25:04,AAPL: Huge rising wedge - likely drop,Short,"\n\n AAPL is in a huge rising wedge. If this bearish pattern is broken, the downside targets for APPL are T1 160 which is 50 days MA, and T2 150 where BigRed aka 200 days SMA. On this level there could be only short-lived bounces. \n\r\nThere are substantial bearish divergences on both RSI and MACD signaling end of rise is near or here. \n\r\nAlso, AAPL stopped on the resistance line, a blue dotted line. \n\r\nAll this is enough for me to take 50% already on the short position and the rest of the position after it breaks the pattern. \n\r\nWith proper risk management tools and stop loss in the right position this almost certainly wins the trade. \n\n\n\n\n",Consistent_Trades,Positive,1,-1,"[aapl, huge, rising, wedge, bearish, pattern, broken, downside, targets, appl, t1, 160, 50, days, t2, 150, bigred, aka, 200, days, sma, level, could, bounces, substantial, bearish, divergences, rsi, macd, signaling, end, rise, near, also, aapl, stopped, resistance, line, blue, dotted, line, enough, take, 50, already, short, position, rest, position, breaks, pattern, proper, risk, management, tools, stop, loss, right, position, almost, certainly, wins, trade]",2023-05-23,...,173.130005,173.380005,171.279999,171.559998,171.328644,50747300.0,171.608261,175.193420,177.010971,177.580200
538,2023-05-23 18:50:40,AAPL: Aapl 🍎,Short,"\n\n Price has formed a megaphone \n\r\n2 options I see for next week, \n\r\n1.we either fake up 1 more time to resistance around 177 then head lower \n\r\n2. The pullback starts right away and we head back to trendline support around 170.\n \n",ContraryTrader,Positive,1,-1,"[price, formed, megaphone, 2, options, see, next, week, either, fake, 1, time, resistance, around, 177, head, lower, pullback, starts, right, away, head, back, trendline, support, around, 170]",2023-05-23,...,173.130005,173.380005,171.279999,171.559998,171.328644,50747300.0,171.608261,175.193420,177.010971,177.580200
536,2023-05-24 03:50:24,AAPL: Bank of America: Crypto markets have limited upside in the near,N/A,"Bank of America (BAC) said in a research note on Friday that the cryptocurrency market has limited upside in the short term. Analysts Alkesh Shah and Andrew Moss wrote that low market confidence, limited catalysts and outperformance year-to-date have kept the digital asset industry stuck in a trading range, while a challenging macro backdrop may limit digital asset upside. Conversations with clients suggest that hedge funds are returning to token trading, with momentum strategies likely to benefit in part from heightened volatility due to lower trading volumes, the bank said. The report added that traditional finance (TradFi) firms and tech firms continue to build blockchain applications, focusing on the tokenization of demand deposits, repo settlement, and bond issuance.",pangj430,Positive,1,0,"[bank, america, bac, said, research, note, friday, cryptocurrency, market, limited, upside, short, term, analysts, alkesh, shah, andrew, moss, wrote, low, market, confidence, limited, catalysts, outperformance, kept, digital, asset, industry, stuck, trading, range, challenging, macro, backdrop, may, limit, digital, asset, upside, conversations, clients, suggest, hedge, funds, returning, token, trading, momentum, strategies, likely, benefit, part, heightened, volatility, due, lower, trading, volumes, bank, said, report, added, traditional, finance, tradfi, firms, tech, firms, continue, build, blockchain, applications, focusing, tokenization, demand, deposits, repo, settlement, bond, issuance]",2023-05-24,...,171.089996,172.419998,170.520004,171.839996,171.608261,45143500.0,172.756714,177.060898,179.847137,180.326492
524,2023-05-24 20:41:36,AAPL: APPLE Will Go UP! Buy!,Lo